# Day 3: Supply II - Enhancing the model

We will continue our modeling exercise by adding carbon taxes, renewable subsidies, and investment to the model.

This will allow us to consider the policy impacts of alternative energy transition policies.

The data and code are based on the paper "The Efficiency and Sectoral Distributional Implications of Large-Scale Renewable Policies," by Mar Reguant.

We first load relevant libraries, same as last session.

In [3]:
#using Pkg
#Pkg.add(["DataFrames", "CSV", "JuMP", "Ipopt", "Plots", "Printf"])


In [4]:
using DataFrames
using CSV
using JuMP
using Ipopt
using Plots
using Printf

Remember to set your path correctly:

## Building the model

We load the same data as last week, and also clean it up to simplify it further and create the demand and import curves.

In [5]:
dfclust = CSV.read("data_jaere_clustered_500.csv", DataFrame);

# Re-scaling (we multiply by 8.76 to make it into a full year of hours (divided by 1000))
dfclust.weights = 8.76 * dfclust.weights / sum(dfclust.weights);

# Here only one demand type to make it easier
dfclust.demand = dfclust.q_residential + dfclust.q_commercial + dfclust.q_industrial;

# Calibrate demand based on elasticities (using 0.1 here as only one final demand)
elas = [.1, .2, .5, .3];
dfclust.b = elas[1] * dfclust.demand ./ dfclust.price;  # slope
dfclust.a = dfclust.demand + dfclust.b .* dfclust.price;  # intercept

# Calibrate imports (using elas 0.3)
dfclust.bm = elas[4] * dfclust.imports ./ dfclust.price;  # slope
dfclust.am = dfclust.imports - dfclust.bm .* dfclust.price;  # intercept

In [26]:
dfclust

,price,imports,q_commercial,q_industrial,q_residential,wind_cap,solar_cap
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,47.4484,9.39537,19.7026,3.88848,12.9976,0.227655,0.669076
2,32.6478,8.60231,10.4654,3.5081,11.7395,0.369588,0.00484255
3,30.2033,7.53693,14.3369,3.22852,9.54468,0.0924152,0.110296
4,42.3501,7.04916,15.1294,6.03778,8.79985,0.210081,0.72881
5,35.9848,5.19763,9.48416,3.44022,9.57441,0.517585,0.40288
6,25.6185,7.50538,13.5399,4.95729,7.51179,0.0998095,0.433466
7,33.8859,8.27999,15.2935,4.91894,6.21909,0.595772,0.00212942
8,54.2729,8.74016,10.9945,3.7498,12.6627,0.486594,0.0185694
9,34.9291,10.0313,14.2391,3.70911,18.1411,0.487055,0.0341828


The technology file now includes the fixed cost of building new power plants (technologies 3-5). Note that we added an additional row for new natural gas plants.

We will use an annualization factor to pro-rate the importance of fixed costs for one year.

In [6]:
tech = CSV.read("data_technology.csv", DataFrame);
afactor = (1 - (1 / (1.05^20.0))) / 0.05;
tech.F = tech.F ./afactor;
tech.F2 = tech.F2 ./afactor;
tech



,techname,heatrate,heatrate2,F,capLB,capUB,new,renewable
,String15,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,Hydro/Nuclear,10.0,0.0,0.0,1.0,1.0,0,0
2,Existing 1,6.67199,0.0929123,0.0,11.5,11.5,0,0
3,Existing 2,9.79412,0.286247,0.0,14.5,14.5,0,0
4,Existing 3,13.8181,20.5352,0.0,0.578,0.578,0,0
5,New Gas,6.6,0.0,78.4773,0.0,100.0,1,0
6,Wind,0.0,0.0,100.303,0.0,100.0,1,1
7,Solar,0.0,0.0,100.303,0.0,100.0,1,1


In [12]:
describe(tech)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,techname,,Existing 1,,Wind,0,String15
2,heatrate,6.69775,0.0,6.67199,13.8181,0,Float64
3,heatrate2,2.98776,0.0,0.0,20.5352,0,Float64
4,F,39.8691,0.0,0.0,100.303,0,Float64
5,capLB,3.93971,0.0,0.578,14.5,0,Float64
6,capUB,46.7969,0.578,14.5,100.0,0,Float64
7,new,0.428571,0,0.0,1,0,Int64
8,renewable,0.285714,0,0.0,1,0,Int64
9,solar,0.142857,0,0.0,1,0,Int64


### Adding investment

We are now ready to clear the market. We will **maximize welfare** using a non-linear solver.

$$ \max \ CS - Costs - Fixed Costs \\

\text{s.t.} \ \text{operational constraints, market clearing}. $$

Notice that we added the fixed costs to the problem, as we will be solving for the optimal level of wind and solar investment.

In [10]:
## Clear market based on cost minimization
function clear_market_invest(data::DataFrame, tech::DataFrame)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    
    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    
    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] - ) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

clear_market_invest (generic function with 1 method)

In [11]:
results_min = clear_market_invest(dfclust, tech)

Dict{String, Any} with 10 entries:
  "avg_price" => 33.4541
  "solar_gw"  => 0.0
  "cost"      => [682.383, 416.599, 429.477, 637.056, 425.05, 428.07, 453.645, …
  "price"     => [39.8678, 34.0552, 27.078, 40.2672, 35.1904, 34.512, 35.3557, …
  "gas_gw"    => 1.96212
  "status"    => "LOCALLY_SOLVED"
  "imports"   => [8.94505, 8.71356, 7.30296, 6.94515, 5.16321, 8.28702, 8.38773…
  "demand"    => [37.1733, 25.6022, 27.3906, 30.1144, 22.5485, 25.1061, 26.3169…
  "quantity"  => [9.18815 11.5 … 4.08069e-8 1.42882e-9; 3.42648 11.5 … 6.05604e…
  "wind_gw"   => 1.45949e-7

## Environmental regulation

We will begin by adding carbon taxes to the model.



In [32]:
function clear_market_invest_tax(data::DataFrame, tech::DataFrame,tax=30.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    #New Variable
    @variable(model, totale[1:T]); #total emissions
    

    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] - tax*totale[t]) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Define emissions 
    @constraint(model, [t=1:T], totale[t] ==
                    sum(tech.e[i] * quantity[t,i] for i=1:I))

        # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        totale = JuMP.value.(totale);
        avg_emissions = sum(totale[t] * data.weights[t]/sum(data.weights) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_emissions" => avg_emissions,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "totale" => totale,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

clear_market_invest_tax (generic function with 2 methods)

In [33]:
results_tax = clear_market_invest_tax(dfclust, tech)

Dict{String, Any} with 12 entries:
  "avg_price"     => 43.1595
  "price"         => [54.0909, 37.4321, 37.4687, 54.767, 37.5872, 37.8942, 37.4…
  "gas_gw"        => 1.08291
  "status"        => "LOCALLY_SOLVED"
  "totale"        => [6.38951, 4.00592, 4.04652, 6.75805, 4.17773, 4.51784, 4.0…
  "quantity"      => [9.18815 11.5 … 1.09443 6.57676e-9; 3.42648 10.0696 … 1.77…
  "solar_gw"      => 0.0
  "imports"       => [9.78995, 8.98049, 8.08083, 7.66919, 5.26706, 8.58429, 8.5…
  "demand"        => [36.0765, 25.3362, 26.4579, 29.0884, 22.3986, 24.7627, 26.…
  "avg_emissions" => 4.44261
  "cost"          => [617.449, 366.753, 399.339, 576.531, 350.595, 409.634, 367…
  "wind_gw"       => 4.80741

Finally, we will add a subsidy to the model. One can think of the subsidy as a negative cost to renewable power. 

In [36]:
function clear_market_invest_subsidy(data::DataFrame, tech::DataFrame,subsidy=10.0)

    # We declare a model
    model = Model(
        optimizer_with_attributes(
            Ipopt.Optimizer, 
                "print_level"=>0)
        );

    # Set useful indexes
    I = nrow(tech);  # number of techs
    T = nrow(data);  # number of periods

    # Variables to solve for
    @variable(model, price[1:T]);
    @variable(model, demand[1:T]);
    @variable(model, imports[1:T]);
    @variable(model, quantity[1:T, 1:I] >= 0);
    @variable(model, costs[1:T]);
    @variable(model, gross_surplus[1:T]);
    @variable(model, gas_gw >= 0.0);
    @variable(model, wind_gw >= 0.0);
    @variable(model, solar_gw >= 0.0);

    #New Variable
    @variable(model, totale[1:T]); #total emissions
    
    # Maximize welfare including imports costs
    @NLobjective(model, Max, sum(data.weights[t] * 
                (gross_surplus[t] - costs[t] ) for t=1:T)
             - tech.F[5]*gas_gw  - tech.F[6]*wind_gw - tech.F[7]*solar_gw 
             - tech.F2[5]*gas_gw^2  - tech.F2[6]*wind_gw^2 - tech.F2[7]*solar_gw^2);

    # Market clearing
    @constraint(model, [t=1:T], 
        demand[t] == data.a[t] - data.b[t] * price[t]);
    @constraint(model, [t=1:T], 
        imports[t] == data.am[t] + data.bm[t] * price[t]);
    @constraint(model, [t=1:T], 
        demand[t] == sum(quantity[t,i] for i=1:I) + imports[t]);

    # Define surplus
    @constraint(model, [t=1:T], gross_surplus[t]==
                (data.a[t] - demand[t]) * demand[t] / data.b[t] 
            + demand[t]^2/(2*data.b[t]));

    # Define cost
    @constraint(model, [t=1:T], costs[t] ==
                    sum(tech.c[i] * quantity[t,i]
                    + tech.c2[i] * quantity[t,i]^2/2 
                    - subsidy * tech.renewable[i] * quantity[t,i] for i=1:I)
        + (imports[t] - data.am[t])^2/(2 * data.bm[t]))

    # Define emissions 
    @constraint(model, [t=1:T], totale[t] ==
                    sum(tech.e[i] * quantity[t,i] for i=1:I))


        # Constraints on output
    @constraint(model, [t=1:T], 
        quantity[t,1] <= data.hydronuc[t]);
    @constraint(model, [t=1:T,i=2:4], 
        quantity[t,i] <= tech[i,"capUB"]);
    @constraint(model, [t=1:T], 
        quantity[t,5] <= gas_gw);
    @constraint(model, [t=1:T], 
        quantity[t,6] <= wind_gw * data.wind_cap[t]);
    @constraint(model, [t=1:T], 
        quantity[t,7] <= solar_gw * data.solar_cap[t]);


    # Solve model
    optimize!(model);

    status = @sprintf("%s", JuMP.termination_status(model));

    if (status=="LOCALLY_SOLVED")
        p = JuMP.value.(price);
        avg_price = sum(p[t] * data.weights[t]/sum(data.weights) for t=1:T);
        q = JuMP.value.(quantity);
        imp = JuMP.value.(imports);
        d = JuMP.value.(demand);
        cost = JuMP.value.(costs);
        totale = JuMP.value.(totale);
        avg_emissions = sum(totale[t] * data.weights[t]/sum(data.weights) for t=1:T);
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)),
            "avg_price" => avg_price,
            "avg_emissions" => avg_emissions,
            "price" => p,
            "quantity" => q,
            "imports" => imp,
            "demand" => d,
            "cost" => cost,
            "totale" => totale,
            "gas_gw" => JuMP.value.(gas_gw),
            "wind_gw" => JuMP.value.(wind_gw),
            "solar_gw" => JuMP.value.(solar_gw));
        return results
    else
        results = Dict("status" => @sprintf("%s",JuMP.termination_status(model)));
        return results
    end

end

clear_market_invest_subsidy (generic function with 2 methods)

In [37]:
results_subsidy = clear_market_invest_subsidy(dfclust, tech)

Dict{String, Any} with 12 entries:
  "avg_price"     => 32.7805
  "price"         => [39.7889, 29.318, 30.0876, 40.2601, 30.2362, 34.9357, 30.5…
  "gas_gw"        => 0.996083
  "status"        => "LOCALLY_SOLVED"
  "totale"        => [7.49409, 4.49075, 4.49075, 7.75092, 4.49075, 4.84853, 4.4…
  "quantity"      => [9.18815 11.5 … 1.05553 4.88794e-9; 3.42648 11.5 … 1.71361…
  "solar_gw"      => 0.0
  "imports"       => [8.94036, 8.3391, 7.52827, 6.94479, 4.94853, 8.32427, 8.03…
  "demand"        => [37.1794, 25.9753, 27.1204, 30.115, 22.8582, 25.063, 26.68…
  "avg_emissions" => 5.14646
  "cost"          => [645.482, 364.573, 409.749, 603.99, 339.42, 416.4, 354.061…
  "wind_gw"       => 4.63653

## Follow-up exercises

1. Consider a tax and a subsidy that reach the same target of emissions. What are the different costs? How are the different components of welfare affected?

Note: This will require you to include emissions as an input or an output to the function.